In [ ]:
"""
price_per_meter, 
area
GDP, ////
Money Supply (M1), /////////
Domestic Liquidity (M2), ////////
IR,
Real estate ownership (ALEX),
population XXX doesn't matter
points
"""

"\nprice_per_meter, \narea\nGDP, ////\nMoney Supply (M1),\nDomestic Liquidity (M2),\nIR,\nReal estate ownership (ALEX),\npopulation XXX doesn't matter\n"

In [ ]:
import pandas as pd

In [ ]:
gdp = pd.read_csv("/kaggle/input/gdp-egypt-housing/Gross Domestic Product.csv")

In [ ]:
gdp = gdp[gdp["Sector"] == "Total"]

In [ ]:
gdp["Year"] = gdp["Year"].str.split("/").str[0]

In [ ]:
gdp["Year"] = gdp["Year"].astype(int)

In [ ]:
gdp = gdp[gdp["Year"] >= 2017]

In [ ]:
gdp["Year"] = gdp["Year"].astype(str)
gdp["Date"] = gdp["Year"] + gdp["Quarter"]

In [ ]:
gdp = gdp[["Real Estate Activitie", "Date"]]

In [ ]:
gdp = gdp.reset_index(drop=True)

In [172]:
gdp["Date"] = pd.PeriodIndex(gdp["Date"], freq="Q-DEC")

In [177]:
gdp = gdp[['Real Estate Activitie', "Date"]]

In [118]:
dl = pd.read_csv("/kaggle/input/eg-domestic-liquidity/Domestic Liquidity.xlsx - Historical Data.csv")

In [119]:
dl = dl.T.reset_index(drop=True)

dl.columns = dl.iloc[0]
dl = dl[1:].reset_index(drop=True)

dl = dl[list(dl.columns[3:7])]

dl = dl[2:].reset_index(drop=True)

dl["End of"] = dl["End of"].astype(int)
dl = dl[dl["End of"] >= 2017].reset_index(drop=True)

dl.columns = ["Year", "NaN", "NaN", "Month", "Domestic Liquidity", "Money Supply"]

dl = dl.drop(["NaN"], axis=1)

dl["Domestic Liquidity"] = dl["Domestic Liquidity"].astype(int)
dl["Money Supply"] = dl["Money Supply"].astype(int)

In [120]:
def clean_months(month):
    month = month
    if "." not in month:
        return month[:3]
    return month

dl["Month"] = dl["Month"].apply(clean_months)

In [132]:
import calendar

dl["Month"] = dl["Month"].str.replace(r"\.$", "", regex=True)

dl["Month"] = pd.to_datetime(dl["Month"], format="%b", errors="coerce").dt.month

In [137]:
dl["Month"] = dl["Month"].fillna(9).astype(int)

In [140]:
dl["Date"] = pd.to_datetime(dl[["Year", "Month"]].astype(str).agg("-".join, axis=1)).dt.to_period("Q")

In [154]:
dl = dl.groupby("Date")[["Domestic Liquidity", "Money Supply"]].mean().reset_index()

In [155]:
dl = dl.drop(["Year", "Month"], axis=1)

,Date,Domestic Liquidity,Money Supply
0,2017Q1,2.690882e+06,6.291380e+05
1,2017Q2,2.850788e+06,6.819570e+05
2,2017Q3,2.998358e+06,7.185250e+05
3,2017Q4,3.143779e+06,7.299023e+05
4,2018Q1,3.297542e+06,7.513697e+05
5,2018Q2,3.411714e+06,8.001287e+05
6,2018Q3,3.510292e+06,8.309283e+05
7,2018Q4,3.591213e+06,8.309413e+05
8,2019Q1,3.679827e+06,8.451907e+05
9,2019Q2,3.807644e+06,9.046673e+05


In [160]:
alex_prices = pd.read_csv("/kaggle/input/alex-prices-2017-2024/alex_prices_2017_2024.csv", index_col=0)

In [165]:
alex_prices["Date"] = pd.PeriodIndex(alex_prices["Date"], freq="Q-DEC")

In [168]:
alex = pd.merge(alex_prices, dl, on="Date", how="left")

In [179]:
alex = pd.merge(alex, gdp, on="Date", how="left")

In [185]:
print(alex.drop(["Date"], axis=1).corr())

                       Price Per Meter    Points  Real Estate Ownership  \
Price Per Meter               1.000000  0.950928              -0.182363   
Points                        0.950928  1.000000              -0.033140   
Real Estate Ownership        -0.182363 -0.033140               1.000000   
Core (m/m)                    0.054010  0.176421              -0.001543   
Domestic Liquidity            0.359225  0.551140               0.130017   
Money Supply                  0.399309  0.596749               0.125567   
Real Estate Activitie         0.669508  0.762236               0.149870   

                       Core (m/m)  Domestic Liquidity  Money Supply  \
Price Per Meter          0.054010            0.359225      0.399309   
Points                   0.176421            0.551140      0.596749   
Real Estate Ownership   -0.001543            0.130017      0.125567   
Core (m/m)               1.000000            0.417385      0.356152   
Domestic Liquidity       0.417385           

In [190]:
macro = alex.drop(["Date"], axis=1)

In [194]:
macro.dtypes

Price Per Meter          float64
Points                   float64
Real Estate Ownership    float64
Core (m/m)               float64
Domestic Liquidity       float64
Money Supply             float64
Real Estate Activitie    float64
dtype: object

In [195]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(max_iter=10, random_state=0)
df_imputed = pd.DataFrame(imputer.fit_transform(macro), columns=macro.columns)

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [197]:
df_imputed["Date"] = alex["Date"] 

In [199]:
macro = df_imputed.drop(["Price Per Meter"], axis=1)

In [203]:
macro = macro.rename(columns={"Points": "Aqarmap Index", "Real Estate Ownership": "Alexandria's Real Estate Ownership (GDP) (Thousands EGP)",
                     "Core (m/m)": "Inflation Rate", "Real Estate Activitie": "Egypt's Real Estate Activities (GDP) (Millions EGP)"})

In [205]:
macro = macro.iloc[:, [-1] + list(range(len(macro.columns) - 1))]

In [206]:
macro.to_csv("macroeconomics.csv")

In [208]:
macro

,Date,Aqarmap Index,Alexandria's Real Estate Ownership (GDP) (Thousands EGP),Inflation Rate,Domestic Liquidity,Money Supply,Egypt's Real Estate Activities (GDP) (Millions EGP)
0,2017Q1,2664.666667,3.806460e+07,2.858000,2.690882e+06,6.291380e+05,91939.00
1,2017Q2,2522.000000,3.806460e+07,1.627333,2.850788e+06,6.819570e+05,85056.53
2,2017Q3,3046.666667,3.806460e+07,1.089667,2.998358e+06,7.185250e+05,99464.13
3,2017Q4,2926.500000,2.938976e+07,0.548000,3.143779e+06,7.299023e+05,109395.07
4,2018Q1,3000.500000,3.806460e+07,0.421000,3.297542e+06,7.513697e+05,122774.83
5,2018Q2,3246.666667,3.806460e+07,1.416333,3.411714e+06,8.001287e+05,110275.58
6,2018Q3,3206.000000,3.806460e+07,0.366667,3.510292e+06,8.309283e+05,115347.64
7,2018Q4,2734.000000,3.502928e+07,0.469333,3.591213e+06,8.309413e+05,130243.82
8,2019Q1,2541.333333,3.806460e+07,0.614333,3.679827e+06,8.451907e+05,143879.37
9,2019Q2,2693.000000,3.806460e+07,0.625333,3.807644e+06,9.046673e+05,132959.32


In [211]:
alex_prices[["Date", "Price Per Meter"]].to_csv("price_per_meter_2017_2024.csv")